In [74]:
%pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [15]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.serialization import Encoding, ParameterFormat, PublicFormat
from cryptography.hazmat.primitives.serialization import load_pem_parameters, load_pem_public_key
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

### Key exchange

In [16]:
parameters = dh.generate_parameters(generator=2, key_size=1024, backend=default_backend())

with open("shared_file", "wb") as file:
    file.write(parameters.parameter_bytes(encoding=Encoding.PEM, format=ParameterFormat.PKCS3))

In [17]:
with open ("shared_file", "rb") as file:
    public_mixed_with_D = load_pem_public_key(file.read()) 

private_key_C = parameters.generate_private_key() # red
public_mixed_with_C  = private_key_C.public_key()  #orange

with open("shared_file", "wb") as file:
    file.write(public_mixed_with_C.public_bytes(encoding=Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo))

In [18]:
shared_key_D_then_C = private_key_C.exchange(public_mixed_with_D) # brown

derived_key_D_then_C = HKDF(
    algorithm = hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
    backend=default_backend()
).derive(shared_key_D_then_C)

print(derived_key_D_then_C)

NameError: name 'HKDF' is not defined

In [83]:
with open('shared_file', 'rb') as f:
    public_key_d = f.read()
    public_key_d = int.from_bytes(public_key_d, byteorder='big', signed=False)

sC = (public_key_d**c)%P

print(sC)

14


### Encryption

In [2]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_public_key

In [7]:
controler_private_key = rsa.generate_private_key(public_exponent= 65537, key_size= 8192)
controler_public_key = controler_private_key.public_key()

pem = controler_public_key.public_bytes(
   encoding=serialization.Encoding.PEM,
   format=serialization.PublicFormat.SubjectPublicKeyInfo
)

with open('shared_file', 'wb') as f:
    f.write(pem)
    f.close()

In [9]:
with open("shared_file", "rb") as file:
    delentture_public_key = load_pem_public_key(file.read())

In [10]:
with open("data/columns_description.csv", 'r') as file:
  columns_description = file.read()

In [11]:
padding_obj = padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None)

encrypted_message = delentture_public_key.encrypt(columns_description.encode(), padding_obj)

ValueError: Encryption/decryption failed.